In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.ticker import FuncFormatter
import colorcet as cc

In [23]:
model = 'bloom-1b7'
# layer = 'last-layer' 
# layer = 'inter-layer-13' 
layer = 'inter-layer-17' 
drop_points = []

plt.rc('text', usetex=True )
plt.rc('font', family='Times New Roman', weight='heavy', size=14)
palette = sns.color_palette(cc.glasbey)
sns.set_palette(palette)
output_plot_path = 'experiments/heatmaps/through_ckpts'

In [20]:
df = pd.read_csv(f'csv_files/{model}_f1-score.csv', index_col=0).sort_index() * 100
row_titles = ['en', 'ar', 'es', 'eu', 'fr', 'hi', 'mr', 'pt', 'ta', 'ur', 'vi', 'zh']
df = df.reindex(index=row_titles)
df = df.drop(columns=['best'])
# convert the column names to integers
df.columns = df.columns.astype(int)/1000
ax = sns.heatmap(df, annot=True, fmt=".1f", cmap='rocket')
ax.set_xlabel("Global Steps (Thousands)", fontweight='bold')  
ax.set_ylabel("Target Languages", fontweight='bold')
ax.set_xticklabels([int(label) for label in df.columns])
# def format_func(value, tick_number):
#     return f'{int(value/1000)}'
# ax.xaxis.set_major_formatter(FuncFormatter(format_func))

for wanted_label in drop_points:
    wanted_index = df.columns.to_list().index(wanted_label)
    N = len(df.index)
    x, y, w, h = wanted_index, 0, 1, N
    ax.add_patch(Rectangle((x, y), w, h, fill=False, edgecolor='black', lw=2, clip_on=False))
    ax.tick_params(length=0)
# ax.set_title(f"Language-wise Cross-lingual Tranfer Performance\non POS tagging: {model}", fontweight='bold')
plt.savefig(f'{output_plot_path}/{model}_f1_score.pdf', bbox_inches='tight') 
plt.clf()

<Figure size 640x480 with 0 Axes>

In [21]:
df = pd.read_csv(f'csv_files/{model}_xnli_acc.csv', index_col=0).sort_index() * 100
row_titles = ['en', 'ar', 'es', 'fr', 'hi', 'ur', 'vi', 'zh']
# drop_rows = ['bg', 'de', 'el','ru', 'sw', 'th', 'tr']
# df = df.drop(index=drop_rows)
df = df.reindex(index=row_titles)
df.columns = df.columns.astype(int)/1000
ax = sns.heatmap(df, annot=True, fmt=".1f", cmap='rocket')
ax.set_xlabel("Global Steps (Thousands)", fontweight='bold')  
ax.set_ylabel("Target Languages", fontweight='bold')
ax.set_xticklabels([int(label) for label in df.columns])
for wanted_label in drop_points:
    wanted_index = df.columns.to_list().index(wanted_label)
    N = len(df.index)
    x, y, w, h = wanted_index, 0, 1, N
    ax.add_patch(Rectangle((x, y), w, h, fill=False, edgecolor='black', lw=2, clip_on=False))
    ax.tick_params(length=0)
# ax.set_title(f"Language-wise Cross-lingual Tranfer Performance\non XNLI: {model}", fontweight='bold')
plt.savefig(f'{output_plot_path}/{model}_xnli_acc.pdf', bbox_inches='tight') 
plt.clf()

<Figure size 640x480 with 0 Axes>

In [6]:
model_ckpt_dict = {'bloom-560m': ['1000', '100000', '200000', '300000', '400000', '600000'], 
                   'bloom-1b1': ['1000', '10000', '100000', '200000', '300000', '400000', '500000', '600000'], 
                   'bloom-1b7': ['50000', '100000', '150000', '200000']}
models = []
for model, ckpt_list in model_ckpt_dict.items():
    models.append(model)
    for ckpt in ckpt_list:
        models.append(f'{model}-intermediate-global_step{ckpt}')

for model in models:

    df = pd.read_csv(f'csv_files/cos_similarity_by_layers/{model}.csv', index_col=0).T
    print(df)
    ax = sns.heatmap(df, cmap='rocket', linewidth=.5)
    ax.set_xlabel("Target Languages", fontweight='bold')  
    ax.set_ylabel("Layer", fontweight='bold')

    model_size = model.split('-intermediate-global_step')[0]
    if len(model.split('-intermediate-global_step')) == 1:
        ax.set_title(f"Average Parallel Sentence Similarity on\n{model_size}, Present Version", fontweight='bold')
    else:
        model_ckpt = int(int(model.split('-intermediate-global_step')[1])/1000)
        ax.set_title(f"Average Parallel Sentence Similarity on\n{model_size}, Global Step {model_ckpt}k", fontweight='bold')
        
    plt.savefig(f'experiments/heatmaps/cos_similarity_by_layers/{model}.pdf', bbox_inches='tight') 
    plt.clf()

          hi        eu        ur        es        ta        fr        vi  \
1   0.954312  0.953701  0.801495  0.936296  0.964694  0.942533  0.968506   
2   0.965006  0.960423  0.904957  0.958071  0.964261  0.958062  0.968520   
3   0.956453  0.951859  0.904566  0.952653  0.954101  0.948815  0.953469   
4   0.936466  0.931526  0.884715  0.939677  0.923635  0.935336  0.935831   
5   0.902513  0.881511  0.839541  0.917882  0.865254  0.913109  0.896399   
6   0.863212  0.855415  0.793001  0.902278  0.810508  0.894396  0.872836   
7   0.840775  0.838620  0.763903  0.896824  0.787041  0.888342  0.864847   
8   0.831107  0.831157  0.746322  0.892284  0.773522  0.882632  0.859758   
9   0.831196  0.830605  0.750539  0.893295  0.769439  0.884828  0.863109   
10  0.841337  0.840684  0.762845  0.897720  0.773830  0.890185  0.871767   
11  0.844805  0.845711  0.766692  0.898913  0.774857  0.891994  0.875476   
12  0.842436  0.845202  0.760161  0.899341  0.765901  0.893440  0.880241   
13  0.839903

<Figure size 640x480 with 0 Axes>

In [9]:
df = pd.read_csv(f'csv_files/{model}_{layer}_cos-similarity.csv', index_col=0)
print(df)
df = df.drop(columns=['best'])
df.columns = df.columns.astype(int)/1000
ax = sns.heatmap(df, annot=True, fmt=".3f", cmap='rocket')
ax.set_xlabel("Global Steps (K)", fontweight='bold')  
ax.set_ylabel("Target Languages", fontweight='bold')
ax.set_xticklabels([int(label) for label in df.columns])
for wanted_label in drop_points:
    wanted_index = df.columns.to_list().index(wanted_label)
    N = len(df.index)
    x, y, w, h = wanted_index, 0, 1, N
    ax.add_patch(Rectangle((x, y), w, h, fill=False, edgecolor='black', lw=2, clip_on=False))
    ax.tick_params(length=0)
ax.set_title(f"Average Parallel Sentence Similarity\non {model}, Layer {layer.split('-')[-1]}", fontweight='bold')
plt.savefig(f'{output_plot_path}/{model}_{layer}_cos-similarity.pdf', bbox_inches='tight') 
plt.clf()

          best      1000    100000    200000    300000    400000    600000
ar    0.077330  0.457647  0.176615  0.027245  0.678346  0.105271  0.356799
es    0.887743  0.727895  0.889176  0.990906  0.993016  0.996312  0.734460
eu    0.746171  0.703451  0.765071  0.769899  0.927408  0.957210  0.746231
fr    0.894948  0.727756  0.905522  0.992900  0.995207  0.996782  0.737500
hi    0.116905  0.657520  0.358499  0.062266  0.669454 -0.015551  0.745364
pt    0.902158  0.760800  0.875740  0.989301  0.990275  0.997125  0.759483
ta    0.088106  0.577768  0.226207  0.001060  0.158025  0.048556  0.756324
ur    0.129696  0.466826  0.232983 -0.143666  0.574029  0.228214  0.556315
vi    0.865738  0.665003  0.753974  0.928092  0.941562  0.940514  0.773889
code  0.953850  0.135067  0.518225  0.984069  0.990153  0.981627  0.225760


<Figure size 640x480 with 0 Axes>

In [24]:
df = pd.read_csv(f'csv_files/{model}_{layer}_ovlp-rate_POS.csv', index_col=0).sort_index()
df = df.drop(columns=['best'])
df = df.drop(['en', 'ca'])
df.columns = df.columns.astype(int)/1000
ax = sns.heatmap(df, annot=True, cmap='rocket')
ax.set_xlabel("Global Steps (K)", fontweight='bold')  
ax.set_ylabel("Target Languages", fontweight='bold')
ax.set_xticklabels([int(label) for label in df.columns])

for wanted_label in drop_points:
    wanted_index = df.columns.to_list().index(wanted_label)
    N = len(df.index)
    x, y, w, h = wanted_index, 0, 1, N
    ax.add_patch(Rectangle((x, y), w, h, fill=False, edgecolor='black', lw=2, clip_on=False))
    ax.tick_params(length=0)
ax.set_title(f"Language-wise Neuron Overlap Rate\non {model}, Layer {layer.split('-')[-1]}", fontweight='bold')
plt.savefig(f'{output_plot_path}/{model}_{layer}_ovlp-rate.pdf', bbox_inches='tight') 
plt.clf()

<Figure size 640x480 with 0 Axes>